# Continual Learning Strategy Experiments

**Import Packages**  
The following cell will import the required packages, and print the their current version, and indicate how many GPU's are connected to the system.

In [ ]:
# Project Dependencies | Double Click to Expand/Collapse
# Author: Luke Collins 
# Date Created: 2023-09-28
# Date Modified: 2023-09-28
# Description: This file contains the code used in the publication "XX" to select the base_model for this study.

import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import pydot


import matplotlib as mpl
import matplotlib.pyplot as plt

import sklearn as sk
from sklearn.metrics import confusion_matrix, roc_curve, f1_score, auc
from sklearn.metrics import precision_score, recall_score

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import Xception, InceptionV3, ResNet50, VGG16, VGG19 
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess_input
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_v3_preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess_input
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.utils import plot_model

print("Printing versions of libraries used:")
print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Seaborn:", sns.__version__)
print("Pydot:", pydot.__version__)
print("Matplotlib:", mpl.__version__)
print("Scikit-learn version:", sk.__version__)
print("Tensorflow version:", tf.__version__)
print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))



----
## **Notebook Configuration Cell**  
The following cell should always be run directly after the packages are imported, this is where you will configure directories to suit your system if reproducing this experiment at home. Do not proceed if your cells output produces and `false` for any of the directories.

**Warning**   
Before altering path, if you wish the use a different dataset on a specific test, we recommend change the `experiment_train_directory` or `experiment_test_directory` on lines 5-6 of the experiment cell.

In [ ]:
# Project Configuration File | Double Click to Expand/Collapse
# Load Directories
train_directory = "../datasets/train-20"
test_directory = "../datasets/test-20"
experiment_base_directory = "../experiments/continual-learning-strategy"

# Create directories if they don't exist
os.makedirs(experiment_base_directory, exist_ok=True)

# Check if directories exist
print("Train directory exists:", os.path.isdir(train_directory))
print("Test directory exists:", os.path.isdir(test_directory))
print("Experiment directory exists:", os.path.isdir(experiment_base_directory))

----
## **Dataset Creation Utilities**  
The following cell will define the functions required to create the training, validation and test datasets.

In [ ]:
# Project Dataset Creation Utilities | Double Click to Expand/Collapse
def create_train_val_datasets(
    preprocessing_model,
    input_dir,
    img_height=299,
    img_width=299,
    batch_size=32,
    augment_data=False
):
    """
    Creates train and validation datasets from images in the input directory.

    Args:
        preprocessing_model (function): The preprocessing function to apply to the images.
        input_dir (str): The path to the input directory containing 'real' and 'fake' subdirectories.
        img_height (int): The height of the input images. Default is 299.
        img_width (int): The width of the input images. Default is 299.
        batch_size (int): The batch size for the data generators. Default is 32.
    Returns:
        tf.data.Dataset, tf.data.Dataset: The train and validation datasets.
    """
    normalDatagen = ImageDataGenerator(
        preprocessing_function=preprocessing_model,
        validation_split=0.2                # Create a 80/20 train/validation split
    )

    augmentDatagen = ImageDataGenerator(
            preprocessing_function=preprocessing_model,
            rotation_range=10,              # Random rotation within +/- 10 degrees
            width_shift_range=0.1,          # Random horizontal shift within +/- 10% of image width
            height_shift_range=0.1,         # Random vertical shift within +/- 10% of image height
            shear_range=0.1,                # Random shearing
            zoom_range=0.1,                 # Random zooming
            horizontal_flip=True,           # Horizontal flipping
            brightness_range=[0.8, 1.2],    # Random brightness adjustment within the given range
            validation_split=0.2            # Create a 80/20 train/validation split
    )


    datagen = augmentDatagen if augment_data else normalDatagen
    train_gen = datagen.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        subset="training",
        shuffle=True
    )

    val_gen = normalDatagen.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        subset="validation",
        shuffle=True
    )
    
    return train_gen, val_gen

def create_test_dataset(
    preprocessing_model,
    input_dir,
    img_height=299,
    img_width=299,
    batch_size=32,
):
    """
    Creates a test dataset from images in the input directory.

    Args:
        preprocessing_model (function): The preprocessing function to apply to the images.
        input_dir (str): The path to the input directory containing 'real' and 'fake' subdirectories..
        img_height (int): The height of the input images. Default is 299.
        img_width (int): The width of the input images. Default is 299.
        batch_size (int): The batch size for the data generators. Default is 32.
        save_dataset (bool): Whether to save the dataset as a TFRecord file. Default is True.
    Returns:
        tf.data.Dataset: The test dataset.
    """
    datagen = ImageDataGenerator(
        preprocessing_function=preprocessing_model
    )

    return datagen.flow_from_directory(
        input_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True,
    )

----
## **Plotting Utilities**  
The following cell will define the functions required to plot model accuracy on training and validation metrics.

In [ ]:
# Project Plot/Graphic Creation Utilities | Double Click to Expand/Collapse
def create_accuracy_plots(folder_path):
    """
    Creates accuracy plots for each epoch report in the specified folder.

    Args:
        folder_path (str): Path to the folder containing the epoch reports.

    Returns:
        None
    """
    # Get a list of all CSV files in the folder
    epoch_reports = [file for file in os.listdir(folder_path) if file.endswith('epochs_report.csv')]

    # Create the accuracy_plots directory if it doesn't exist
    accuracy_plots_path = os.path.join(folder_path, 'accuracy_plots')
    os.makedirs(accuracy_plots_path, exist_ok=True)

    # Cycle through the epoch reports and create plots
    for report in epoch_reports:
        # Read the epoch history CSV file
        report_path = os.path.join(folder_path, report)
        df = pd.read_csv(report_path)

        # Get the number of epochs
        epochs = range(1, len(df) + 1)

        # Extract the relevant columns
        train_accuracy = df['binary_accuracy']
        val_accuracy = df['val_binary_accuracy']

        # Create the plot
        plt.figure(figsize=(8, 6))
        plt.plot(epochs, train_accuracy, label='Train Accuracy')
        plt.plot(epochs, val_accuracy, label='Validation Accuracy')

        # Set plot title and labels
        plot_name = report.split('_')[0] + '_accuracy_plot.png'
        plt.title(f'Accuracy History - {plot_name}')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')

        # Set plot legend
        plt.legend()

        # Save the plot
        plot_path = os.path.join(accuracy_plots_path, plot_name)
        plt.savefig(plot_path)

        # Display the plot (optional)
        plt.show()

def create_all_metrics_plots(folder_path):
    """
    Creates plots for various metrics (accuracy, precision, recall) for each epoch report in the specified folder.

    Args:
        folder_path (str): Path to the folder containing the epoch reports.

    Returns:
        None
    """
    # Get a list of all CSV files in the folder
    epoch_reports = [file for file in os.listdir(folder_path) if file.endswith('epochs_report.csv')]

    # Create the accuracy_plots directory if it doesn't exist
    accuracy_plots_path = os.path.join(folder_path, 'accuracy_plots')
    os.makedirs(accuracy_plots_path, exist_ok=True)

    # Cycle through the epoch reports and create plots
    for report in epoch_reports:
        # Read the epoch history CSV file
        report_path = os.path.join(folder_path, report)
        df = pd.read_csv(report_path)

        # Get the number of epochs
        epochs = range(1, len(df) + 1)

        # Extract the relevant columns
        train_accuracy = df['binary_accuracy']
        train_precision = df['precision']
        train_recall = df['recall']
        val_accuracy = df['val_binary_accuracy']
        val_precision = df['val_precision']
        val_recall = df['val_recall']

        # Create the plot
        plt.figure(figsize=(8, 6))
        plt.plot(epochs, train_accuracy, label='Train Accuracy')
        plt.plot(epochs, train_precision, label='Train Precision')
        plt.plot(epochs, train_recall, label='Train Recall')
        plt.plot(epochs, val_accuracy, label='Validation Accuracy')
        plt.plot(epochs, val_precision, label='Validation Precision')
        plt.plot(epochs, val_recall, label='Validation Recall')

        # Set plot title and labels
        plot_name = report.split('_')[0] + '_metrics_plot.png'
        plt.title(f'Metrics History - {plot_name}')
        plt.xlabel('Epochs')
        plt.ylabel('Metrics')

        # Set plot legend
        plt.legend()

        # Save the plot
        plot_path = os.path.join(accuracy_plots_path, plot_name)
        plt.savefig(plot_path)

        # Display the plot (optional)
        plt.show()
        
def create_accuracy_loss_plots(folder_path, experimentId):
    """
    Creates plots for various metrics (accuracy, loss) for each epoch report in the specified folder.

    Args:
        folder_path (str): Path to the folder containing the epoch reports.

    Returns:
        None
    """
    # Get a list of all CSV files in the folder
    epoch_reports = [file for file in os.listdir(folder_path) if file.endswith('epochs_report.csv')]

    # Create the accuracy_plots directory if it doesn't exist
    accuracy_plots_path = os.path.join(folder_path, 'accuracy_plots')
    os.makedirs(accuracy_plots_path, exist_ok=True)

    # Cycle through the epoch reports and create plots
    for report in epoch_reports:
        # Read the epoch history CSV file
        report_path = os.path.join(folder_path, report)
        df = pd.read_csv(report_path)

        # Create the figure and subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

        # Plot for the first subplot
        ax1.plot(df['binary_accuracy'], label='Training Accuracy')
        ax1.plot(df['val_binary_accuracy'], label='Validation Accuracy')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Accuracy')
        ax1.set_title('Training and Validation Accuracy')
        ax1.legend()

        # Plot for the second subplot
        ax2.plot(df['loss'], label='Training Loss')
        ax2.plot(df['val_loss'], label='Validation Loss')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Loss')
        ax2.set_title('Training and Validation Loss')
        ax2.legend()

        # Add a title that spans over the two plots
        plot_name = report.split('_')[0] + '_metrics_plot.png'
        fig.suptitle(f'{experimentID}\nTraining History for {plot_name}\n', fontsize=16, fontweight='bold')

        # Adjust the spacing between subplots and the suptitle
        plt.subplots_adjust(top=0.85, wspace=0.3)

        # Save the plot
        plot_path = os.path.join(accuracy_plots_path, f'{plot_name}_report')
        plt.savefig(plot_path)

        # Display the plot (optional)
        plt.show()      

def create_model_image(model, output_dir, name = 'model', extension = 'png'):
    """
    Creates an image of a TensorFlow model's structure and saves it to the specified directory.

    Args:
        model: The TensorFlow model to visualize.
        output_dir: The directory where the model image should be saved.
        name: The name of the model image file. Default is 'model'.
        extension: The file extension for the model image file. Default is 'png'.

    Returns:
        None
    """
    if extension not in ['png', 'svg', 'pdf', 'jpg']:
        throw(f'Invalid file extension: {extension}')
        
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Construct the full path for the output file
    output_file = os.path.join(output_dir, f'{name}.{extension}')

    # Generate and save the model image
    plot_model(model, to_file=output_file, show_shapes=True, show_layer_names=True)

---
# Elastic Weight Consolidation (EWC) class
---

In [ ]:
# Define the Elastic Weight Consolidation (EWC) class
class EWC:
    def __init__(self, model):
        self.model = model
        self.ewc_losses = []

    def compute_ewc_loss(self, prev_weights, fisher_matrix):
        ewc_loss = 0.0
        for i, layer in enumerate(self.model.layers):
            if layer.trainable_weights:
                ewc_loss += tf.reduce_sum(fisher_matrix[i] * tf.square(layer.weights[0] - prev_weights[i]))
        return ewc_loss

    def register_ewc_losses(self, ewc_losses):
        self.ewc_losses = ewc_losses

---
# Experiments
---
## Experiment #CLS-0001 | 
**Experiment ID:** #CLS-0001  
**Experiment Description:** 


In [ ]:
# Experiment ID:CLS-0001
experimentId = "CLS-0001"

# Load Directories (Leave alone unless specifying a different dataset)
experiment_train_directory = train_directory
experiment_test_directory = test_directory
experiment_directory = f"{experiment_base_directory}/{experimentId}"
experiment_results_directory = f"{experiment_directory}/results"
experiment_models_directory = f"{experiment_directory}/models"

# Create directories if they don't exist
os.makedirs(name=experiment_directory, exist_ok=True)
os.makedirs(name=experiment_models_directory, exist_ok=True)
os.makedirs(name=experiment_results_directory, exist_ok=True)

# Declare lists for report generation
accuracy_report = []

# Declare Test Configurations
test_configurations = [
    [ResNet50, resnet50_preprocess_input, 16, True, "../datasets/FF++/train-1"],
    [ResNet50, resnet50_preprocess_input, 16, True, "../datasets/FF++/train-2"],
    [ResNet50, resnet50_preprocess_input, 16, True, "../datasets/FF++/train-3"],
    [ResNet50, resnet50_preprocess_input, 16, True, "../datasets/FF++/train-4"],
    [ResNet50, resnet50_preprocess_input, 16, True, "../datasets/FF++/train-5"],
    [ResNet50, resnet50_preprocess_input, 16, True, "../datasets/FF++/train-6"],
    [ResNet50, resnet50_preprocess_input, 16, True, "../datasets/FF++/train-7"],
    [ResNet50, resnet50_preprocess_input, 16, True, "../datasets/FF++/train-8"],
    [ResNet50, resnet50_preprocess_input, 16, True, "../datasets/FF++/train-9"],
    [ResNet50, resnet50_preprocess_input, 16, True, "../datasets/FF++/train-10"],
    
]

# Declare accuracy_test list
accuracy_test = []

# Define Early Stopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=8
)

# Define Reduce Learning Rate on Plateau Callback
lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=5,
    verbose=1,
)

# Set Loop Counter 
loop_counter = 0
# Loop through all combinations of models and trainable layers
for pretrained_model, preprocessing_function, batch_size, is_augmented, dataset_path in test_configurations:
    loop_counter += 1
    # Clear Session
    K.clear_session()
    
    # Setup Model Check Pointing
    nameSuffix = 'augmented' if is_augmented == True else 'not_augmented'
    model_checkpoint_vl = ModelCheckpoint(
        filepath=f"{experiment_models_directory}/{pretrained_model.__name__}_model_{nameSuffix}_vl.h5",
        monitor='val_loss',
        mode='min',
        verbose=1,
        save_best_only=True
    )
    
    model_checkpoint_ba = ModelCheckpoint(
        filepath=f"{experiment_models_directory}/{pretrained_model.__name__}_model_{nameSuffix}_ba.h5",
        monitor='val_binary_accuracy',
        mode='max',
        verbose=1,
        save_best_only=True
    )
    
    # Create a dataset and preprocess images to suit base model
    train_ds, val_ds = create_train_val_datasets(preprocessing_function, dataset_path, batch_size=batch_size, augment_data=is_augmented)
    test_ds = create_test_dataset(preprocessing_function, experiment_test_directory, batch_size=batch_size)

    # Create Input Layer
    inputs = Input(shape=(299, 299, 3))

    #####################################################################################################################################
    # Load Previous Model and Weights
    #####################################################################################################################################

    # Load Model
    model = load_model(
        f"/home/luke/Development/deepfake-detection/deepfake-detection-code/experiments/base-model-selection/BMS-0004/models/ResNet50_finetuned_model_augmented.h5",
    )
    
    # Create Input Layer
    inputs = Input(shape=(299, 299, 3))

    # Base model is the second layer in the model
    base_model = model.layers[1]

    # Make Base Model Trainable
    base_model.trainable = True

    # Freeze the base model (do not train everything but last 15 layers)
    for layer in base_model.layers[:-26]:
        layer.trainable = False

    # Hookup Previous Model Layers to New Model
    inputs = keras.Input(shape=(299, 299, 3))
    x = base_model(inputs, training=False)
    
    # Traverse the layers and connect them
    for layer in model.layers[2:]:
        x = layer(x)
    
    # Define the model
    model = keras.Model(inputs=inputs, outputs=x)

    model.summary()
    
    # Compile the model
    model.compile(
        optimizer=Adam(1e-4),  
        loss=BinaryCrossentropy(from_logits=False),
        metrics=[BinaryAccuracy(), Precision(), Recall(), AUC()]
    )

    # Initialize EWC
    ewc = EWC(model)

    #####################################################################################################################################
    # Begin Fine Tuning Entire Model with EWC
    #####################################################################################################################################

    # Start Timer to measure processing time 
    start_time = time.time()

    # Train the model
    history = model.fit(
        train_ds, 
        steps_per_epoch=train_ds.samples // train_ds.batch_size,
        epochs=15, 
        validation_data=test_ds,
        validation_steps=test_ds.samples // test_ds.batch_size,
        callbacks=[early_stopping, lr_reducer, model_checkpoint_vl, model_checkpoint_ba]
    )

    # Calculate Fisher Matrix
    fisher_matrix = []
    for layer in model.layers:
        if layer.trainable_weights:
            fisher_matrix.append(ewc.compute_fisher_matrix(layer))

    # Compute EWC Loss
    ewc_loss = ewc.compute_ewc_loss(prev_weights, fisher_matrix)
    
    # Add EWC Loss to the model's total loss
    model.add_loss(ewc_loss)

    # End Timer and calculate processing time
    end_time = time.time()
    processing_time = end_time - start_time

    # Evaluate the model
    evaluate = model.evaluate(test_ds, steps=test_ds.samples // test_ds.batch_size, verbose=1)

    # Append results to accuracy_test list
    history_key = list(history.history.keys()) # As names of metric names can be dynamic, we need to get the keys of the history object
    accuracy_test.append({
        'Base Model': pretrained_model.__name__,
        'duration': round(processing_time, 3),
        'test_accuracy': round(evaluate[1], 3),
        'test_precision': round(evaluate[2], 3),
        'test_recall': round(evaluate[3], 3),
        'test_loss': round(evaluate[0], 3),
        'train_accuracy': round(history.history[history_key[1]][-1], 3),
        'train_precision': round(history.history[history_key[2]][-1], 3),
        'train_recall': round(history.history[history_key[3]][-1], 3),
        'train_auc': round(history.history[history_key[4]][-1], 3),
        'train_loss': round(history.history[history_key[0]][-1], 3),
        'val_accuracy': round(history.history[history_key[6]][-1], 3),
        'val_precision': round(history.history[history_key[7]][-1], 3),
        'val_recall': round(history.history[history_key[8]][-1], 3),
        'val_auc': round(history.history[history_key[9]][-1], 3),
        'val_loss': round(history.history[history_key[5]][-1], 3)
    })

    epochs_report = pd.DataFrame(history.history)
    epochs_report.to_csv(f"{experiment_results_directory}/{pretrained_model.__name__}_{loop_counter}_epochs_report.csv", index=False)

    model.save(f"{experiment_models_directory}/{pretrained_model.__name__}_finetuned_model_{nameSuffix}.h5")

# Create DataFrame with accuracy score and export as .csv
accuracy_test_results = pd.DataFrame(accuracy_test)
accuracy_test_results.to_csv(f'{experiment_results_directory}/accuracy_test_results.csv', index=False)

# Generate graphs from epochs_reports.csv and output to results folder.
create_accuracy_plots(f"{experiment_results_directory}")
create_all_metrics_plots(f"{experiment_results_directory}")

---
## Experiment #CLS-0002 | 
**Experiment ID:** #TMS-0002  
**Experiment Description:** 


In [ ]:
# Experiment ID:CLS-0002
experimentId = "CLS-0002"

# Load Directories (Leave alone unless specifying a different dataset)
experiment_train_directory = train_directory
experiment_test_directory = test_directory
experiment_directory = f"{experiment_base_directory}/{experimentId}"
experiment_results_directory = f"{experiment_directory}/results"
experiment_models_directory = f"{experiment_directory}/models"

# Create directories if they don't exist
os.makedirs(name=experiment_directory, exist_ok=True)
os.makedirs(name=experiment_models_directory, exist_ok=True)
os.makedirs(name=experiment_results_directory, exist_ok=True)

# Declare lists for report generation
accuracy_report = []

# Declare Test Configurations
test_configurations = [
    [ResNet50, resnet50_preprocess_input, 16, True, "../datasets/Celeb-DF/train-10/"],
    
]

# Declare accuracy_test list
accuracy_test = []

# Define Early Stopping Callback
early_stopping = EarlyStopping(
    monitor='val_loss', 
    mode='min', 
    verbose=1, 
    patience=8
)

# Define Reduce Learning Rate on Plateau Callback
lr_reducer = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=5,
    verbose=1,
)

# Set Loop Counter 
loop_counter = 0
# Loop through all combinations of models and trainable layers
for pretrained_model, preprocessing_function, batch_size, is_augmented, dataset_path in test_configurations:
    loop_counter += 1
    # Clear Session
    K.clear_session()
    
    # Setup Model Check Pointing
    nameSuffix = 'augmented' if is_augmented == True else 'not_augmented'
    model_checkpoint_vl = ModelCheckpoint(
        filepath=f"{experiment_models_directory}/{pretrained_model.__name__}_model_{nameSuffix}_vl.h5",
        monitor='val_loss',
        mode='min',
        verbose=1,
        save_best_only=True
    )
    
    model_checkpoint_ba = ModelCheckpoint(
        filepath=f"{experiment_models_directory}/{pretrained_model.__name__}_model_{nameSuffix}_ba.h5",
        monitor='val_binary_accuracy',
        mode='max',
        verbose=1,
        save_best_only=True
    )
    
    # Create a dataset and preprocess images to suit base model
    train_ds, val_ds = create_train_val_datasets(preprocessing_function, dataset_path, batch_size=batch_size, augment_data=is_augmented)
    test_ds = create_test_dataset(preprocessing_function, experiment_test_directory, batch_size=batch_size)

    # Create Input Layer
    inputs = Input(shape=(299, 299, 3))

    #####################################################################################################################################
    # Load Previous Model and Weights
    #####################################################################################################################################

    # Load Model
    model = load_model(
        f"/home/luke/Development/deepfake-detection/deepfake-detection-code/experiments/base-model-selection/BMS-0004/models/ResNet50_finetuned_model_augmented.h5",
    )
    
    # Create Input Layer
    inputs = Input(shape=(299, 299, 3))

    # Base model is the second layer in the model
    base_model = model.layers[1]

    # Make Base Model Trainable
    base_model.trainable = True

    # Freeze the base model (do not train everything but last 15 layers)
    for layer in base_model.layers[:-26]:
        layer.trainable = False

    # Hookup Previous Model Layers to New Model
    inputs = keras.Input(shape=(299, 299, 3))
    x = base_model(inputs, training=False)
    
    # Traverse the layers and connect them
    for layer in model.layers[2:]:
        x = layer(x)
    
    # Define the model
    model = keras.Model(inputs=inputs, outputs=x)

    model.summary()
    
    # Compile the model
    model.compile(
        optimizer=Adam(1e-4),  
        loss=BinaryCrossentropy(from_logits=False),
        metrics=[BinaryAccuracy(), Precision(), Recall(), AUC()]
    )

    # Initialize EWC
    ewc = EWC(model)

    #####################################################################################################################################
    # Begin Fine Tuning Entire Model with EWC
    #####################################################################################################################################

    # Start Timer to measure processing time 
    start_time = time.time()

    # Train the model
    history = model.fit(
        train_ds, 
        steps_per_epoch=train_ds.samples // train_ds.batch_size,
        epochs=15, 
        validation_data=test_ds,
        validation_steps=test_ds.samples // test_ds.batch_size,
        callbacks=[early_stopping, lr_reducer, model_checkpoint_vl, model_checkpoint_ba]
    )

    # Calculate Fisher Matrix
    fisher_matrix = []
    for layer in model.layers:
        if layer.trainable_weights:
            fisher_matrix.append(ewc.compute_fisher_matrix(layer))

    # Compute EWC Loss
    ewc_loss = ewc.compute_ewc_loss(prev_weights, fisher_matrix)
    
    # Add EWC Loss to the model's total loss
    model.add_loss(ewc_loss)

    # End Timer and calculate processing time
    end_time = time.time()
    processing_time = end_time - start_time

    # Evaluate the model
    evaluate = model.evaluate(test_ds, steps=test_ds.samples // test_ds.batch_size, verbose=1)

    # Append results to accuracy_test list
    history_key = list(history.history.keys()) # As names of metric names can be dynamic, we need to get the keys of the history object
    accuracy_test.append({
        'Base Model': pretrained_model.__name__,
        'duration': round(processing_time, 3),
        'test_accuracy': round(evaluate[1], 3),
        'test_precision': round(evaluate[2], 3),
        'test_recall': round(evaluate[3], 3),
        'test_loss': round(evaluate[0], 3),
        'train_accuracy': round(history.history[history_key[1]][-1], 3),
        'train_precision': round(history.history[history_key[2]][-1], 3),
        'train_recall': round(history.history[history_key[3]][-1], 3),
        'train_auc': round(history.history[history_key[4]][-1], 3),
        'train_loss': round(history.history[history_key[0]][-1], 3),
        'val_accuracy': round(history.history[history_key[6]][-1], 3),
        'val_precision': round(history.history[history_key[7]][-1], 3),
        'val_recall': round(history.history[history_key[8]][-1], 3),
        'val_auc': round(history.history[history_key[9]][-1], 3),
        'val_loss': round(history.history[history_key[5]][-1], 3)
    })

    epochs_report = pd.DataFrame(history.history)
    epochs_report.to_csv(f"{experiment_results_directory}/{pretrained_model.__name__}_{loop_counter}_epochs_report.csv", index=False)

    model.save(f"{experiment_models_directory}/{pretrained_model.__name__}_finetuned_model_{nameSuffix}.h5")

# Create DataFrame with accuracy score and export as .csv
accuracy_test_results = pd.DataFrame(accuracy_test)
accuracy_test_results.to_csv(f'{experiment_results_directory}/accuracy_test_results.csv', index=False)

# Generate graphs from epochs_reports.csv and output to results folder.
create_accuracy_plots(f"{experiment_results_directory}")
create_all_metrics_plots(f"{experiment_results_directory}")